In [2]:
import pandas as pd
import os
from enum import Enum
from typing import List

class MovieDataLoader:
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.extension = os.path.splitext(file_path)[1]
        self.dataframe = None

    def load(self) -> pd.DataFrame:
        self.dataframe = pd.read_csv(
            self.file_path,
            sep='\t' if self.extension == '.txt' else ','
        )
        return self.dataframe

class DB(Enum):
    CASTINGS = 0
    COUNTRIES = 1
    GENRES = 2
    RATES = 3
    MOVIES = 4
    PEOPLES = 5

def get_dataframes() -> List[pd.DataFrame]:
    base_path = "../data/kmrd/kmr_dataset/datafile/kmrd-small"
    db_name = ["castings.csv", "countries.csv", "genres.csv", "rates.csv", "movies.txt", "peoples.txt"]
    db_paths = [os.path.join(base_path, name) for name in db_name]
    return [MovieDataLoader(path).load() for path in db_paths]

In [3]:
dataframes = get_dataframes()
castings_df = dataframes[DB.CASTINGS.value]
countries_df = dataframes[DB.COUNTRIES.value]
genres_df = dataframes[DB.GENRES.value]
rates_df = dataframes[DB.RATES.value]
movies_df = dataframes[DB.MOVIES.value]
peoples_df = dataframes[DB.PEOPLES.value]

print("castings_df")
print(castings_df.head())
print("countries_df")
print(countries_df.head())
print("genres_df")
print(genres_df.head())

castings_df
   movie  people  order  leading
0  10001    4374      1        1
1  10001     178      2        1
2  10001    3241      3        1
3  10001   47952      4        1
4  10001   47953      5        0
countries_df
   movie country
0  10001    이탈리아
1  10001     프랑스
2  10002      미국
3  10003      미국
4  10004      미국
genres_df
   movie   genre
0  10001     드라마
1  10001  멜로/로맨스
2  10002      SF
3  10002     코미디
4  10003      SF


In [4]:
print("rates_df")
print(rates_df.head())
print("movies_df")
print(movies_df.head())
print("peoples_df")
print(peoples_df.head())

rates_df
   user  movie  rate        time
0     0  10003     7  1494128040
1     0  10004     7  1467529800
2     0  10018     9  1513344120
3     0  10021     9  1424497980
4     0  10022     7  1427627340
movies_df
   movie                 title                           title_eng    year  \
0  10001                시네마 천국              Cinema Paradiso , 1988  2013.0   
1  10002              빽 투 더 퓨쳐           Back To The Future , 1985  2015.0   
2  10003            빽 투 더 퓨쳐 2    Back To The Future Part 2 , 1989  2015.0   
3  10004            빽 투 더 퓨쳐 3  Back To The Future Part III , 1990  1990.0   
4  10005  스타워즈 에피소드 4 - 새로운 희망                    Star Wars , 1977  1997.0   

     grade  
0   전체 관람가  
1  12세 관람가  
2  12세 관람가  
3   전체 관람가  
4       PG  
peoples_df
   people    korean        original
0       5    아담 볼드윈    Adam Baldwin
1       8   애드리안 라인     Adrian Lyne
2       9     에이단 퀸     Aidan Quinn
3      13  구로사와 아키라  Akira Kurosawa
4      15     알 파치노       Al Pacino


In [5]:
from datetime import datetime

# 결측치 확인
def print_missing_values():
    print("결측치 확인:")
    print(movies_df.isnull().sum())
    print(castings_df.isnull().sum())
    print(countries_df.isnull().sum())
    print(genres_df.isnull().sum())
    print(rates_df.isnull().sum())

def remove_missing_values():
    global movies_df, peoples_df
    movies_df['year'] = pd.to_numeric(movies_df['year'], errors='coerce')
    year_median = movies_df['year'].median() if not movies_df['year'].isna().all() else 0

    # Fill missing values
    movies_df.fillna({
        'title': 'Unknown',
        'title_eng': 'Unknown',
        'year': year_median,
        'grade': "Unknown",
    }, inplace=True)

    peoples_df.fillna({
        'original': 'Unknown'
    }, inplace=True)

def remove_outliers():
    global castings_df, countries_df, genres_df, rates_df, movies_df
    current_year = datetime.now().year

    castings_df = castings_df[
        (castings_df["leading"] == 0)
        | (castings_df["leading"] == 1)
    ]
    countries_df = countries_df[
        (10000 < countries_df["movie"])
        & (countries_df["movie"] < 11000)
    ]
    genres_df = genres_df[
        (10000 < genres_df["movie"])
        & (genres_df["movie"] < 11000)
    ]
    rates_df = rates_df[
        (0 < rates_df["rate"])
        & (rates_df["rate"] <= 10)
        & (10000 < rates_df["movie"])
        & (rates_df["movie"] < 11000)
    ]
    movies_df = movies_df[
        (movies_df["year"] >= 1895)
        & (movies_df["year"] < current_year)
        & (10000 < movies_df["movie"])
        & (movies_df["movie"] < 11000)
    ]

print_missing_values()

결측치 확인:
movie          0
title          7
title_eng      8
year         390
grade         42
dtype: int64
movie      0
people     0
order      0
leading    0
dtype: int64
movie      0
country    0
dtype: int64
movie    0
genre    0
dtype: int64
user     0
movie    0
rate     0
time     0
dtype: int64


In [6]:
remove_missing_values()
remove_outliers()
print_missing_values()

결측치 확인:
movie        0
title        0
title_eng    0
year         0
grade        0
dtype: int64
movie      0
people     0
order      0
leading    0
dtype: int64
movie      0
country    0
dtype: int64
movie    0
genre    0
dtype: int64
user     0
movie    0
rate     0
time     0
dtype: int64


In [7]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

# D2Coding 폰트 경로 설정
font_path = "../D2Coding.ttf"
fontprop = fm.FontProperties(fname=font_path)

# 폰트 설정
plt.rc("font", family=fontprop.get_name())
plt.rcParams.update({"font.size": 6})  # Set the font size to 14
plt.figure(figsize=(16, 9))


<Figure size 1600x900 with 0 Axes>

<Figure size 1600x900 with 0 Axes>

정답과 예측값이 비슷한지 파악하는 방법은 정답에서 예측값을 뺀 차이를 통해 측정할 수 있다. 즉, 차이가 작을 수록 정답에 근접한 것으로 볼 수 있다.

## 평균 절대 오차 (Mean Absolute Error - MAE)
$MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|$

- 실제 값과 예측 값 간의 절대적인 오차의 평균을 나타내는 지표이다.
- 오차를 모두 절대값으로 변환하고 평균을 계산한다

## 평균 제곱 오차 (Mean Squared Error - MSE) 
$MSE = \frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2$

- 실제 값과 예측 값 간의 제곱 오차의 평균을 나타내는 지표이다.
- 오차를 제곱하여 계산하므로 큰 오차에 더 많은 가중치를 둔다.

## 평균 제곱근 오차 (Root Mean Squared Error - RMSE)
$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}$

- MSE의 제곱근으로, 오차의 크기를 원래 단위로 되돌린다.

## 평균 절대 비율 오차 (Mean Absolute Percentage Error - MAPE)
$MAPE = \frac{100}{n}\sum_{i=1}^n |\frac{y_i - \hat{y}_i}{y_i}|$

- 실제 값과 예측 값 간의 상대적인 오차를 평가하는 지표이다.

In [8]:
import random

class RandomRecommender:
    def __init__(self, rates_df: pd.DataFrame, movies_df: pd.DataFrame):
        self.rates_df = rates_df
        self.movies_df = movies_df

    def run(self, n: int) -> pd.DataFrame:
        self.movies_df['rate_random'] = [random.uniform(1, 10) for _ in range(len(self.movies_df))]
        result_df = self.movies_df.sort_values(by='rate_random', ascending=False).head(n)
        return result_df

class Analyzer:
    def __init__(self, result_df: pd.DataFrame, rates_df: pd.DataFrame):
        self.result_df = result_df
        self.rates_df = rates_df

    def analyze(self):
        merged_df = pd.merge(self.result_df, self.rates_df, on='movie')
        y_true = merged_df['rate']
        y_pred = merged_df['rate_random']

        mae = sum(abs(y_true - y_pred)) / len(y_true)
        mse = sum((y_true - y_pred) ** 2) / len(y_true)
        rmse = mse ** 0.5
        mape = (sum(abs((y_true - y_pred) / y_true)) / len(y_true)) * 100

        return {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'MAPE': mape
        }

recommender = RandomRecommender(rates_df, movies_df)
top_n_recommendations = recommender.run(n=10)

# 결과 분석
analyzer = Analyzer(top_n_recommendations, rates_df)
analysis_results = analyzer.analyze()

# 결과 출력
print(top_n_recommendations)
print("\n분석 결과:")
for metric, value in analysis_results.items():
    print(f"{metric}: {value}")

     movie          title                                  title_eng    year  \
645  10646            도어즈                           The Doors , 1991  1993.0   
603  10604  카사블랑카여, 다시 한번                  Play It Again, Sam , 1972  1989.0   
71   10072           대부 2  Mario Puzo's The Godfather Part II , 1974  2010.0   
493  10494            여학생           L'Etudiante , The Student , 1988  1989.0   
741  10742         원초적 본능                      Basic Instinct , 1992  1992.0   
159  10160     007 카지노 로얄                       Casino Royale , 1967  1971.0   
826  10827            영심이                     0심이: Young-Shim , 1990  1990.0   
288  10289         부베의 여인    La Ragazza Di Bube , Bebo's Girl , 1963  1989.0   
925  10926     90 벌레먹은 장미                                       1990  1990.0   
716  10717             특근                         After Hours , 1985  1989.0   

        grade  rate_random  
645  청소년 관람불가     9.988516  
603        PG     9.962844  
71   청소년 관람불가     9.956616  
493